In [2]:
import pickle
import numpy as np
import os
import csv
import matplotlib.pyplot as plt

# ================= 1. 强力配置 (Stable Config) =================
DATA_DIR = './data'
TRAIN_PATH = os.path.join(DATA_DIR, 'train_data.pkl')
TEST_PATH = os.path.join(DATA_DIR, 'test_data.pkl')

# 策略：宽而浅的网络 + 梯度裁剪 + 强正则化
HIDDEN_DIM = 1024       # 增加宽度 (Wide)
NUM_CLASSES = 5
LEARNING_RATE = 0.0005  # 降低学习率，防止爆炸
BATCH_SIZE = 32         # 减小 Batch Size，增加更新频率
EPOCHS = 40
NUM_MODELS = 5          # 5折集成
L2_REG = 0.01           # 强正则化
GRAD_CLIP = 1.0         # 梯度裁剪阈值 (关键!)

np.random.seed(42)
print(">>> 环境配置完成。启用 'Stable Ultimate' 模式。")

# ================= 2. 数据工具 =================
def load_and_process_data(path, has_labels=True, target_max_size=64):
    if not os.path.exists(path): raise FileNotFoundError(f"{path} 未找到")
    
    with open(path, 'rb') as f: data = pickle.load(f)
    images = np.array(data['images'])
    
    if images.shape[-1] == 3: images = images.transpose(0, 3, 1, 2)
    
    N, C, H, W = images.shape
    # 保持原图比例，最大边长限制在 64，保留特征
    h_step = max(1, H // target_max_size)
    w_step = max(1, W // target_max_size)
    images_processed = images[:, :, ::h_step, ::w_step]
    
    # 归一化 [0, 1]
    X = images_processed.astype(float) / 255.0
    X_flat = X.reshape(N, -1)
    
    y = np.array(data['labels']) if has_labels else None
    return X_flat, y, images_processed.shape[1:]

def augment_batch(X_batch, img_shape):
    """数据增强: 翻转 + 像素偏移"""
    N = X_batch.shape[0]
    C, H, W = img_shape
    
    X_img = X_batch.reshape(N, C, H, W)
    
    # 1. 水平翻转 (50%)
    if np.random.rand() > 0.5:
        X_img = X_img[:, :, :, ::-1]
        
    # 2. 垂直翻转 (20% - 视网膜图像有时是倒置的)
    if np.random.rand() > 0.8:
        X_img = X_img[:, :, ::-1, :]
        
    return X_img.reshape(N, -1)

# ================= 3. 核心算法 (手写 Adam + Clip + Leaky ReLU) =================

def leaky_relu(x, alpha=0.01):
    return np.maximum(alpha * x, x)

def softmax(x):
    # 数值稳定 Softmax
    shift_x = x - np.max(x, axis=1, keepdims=True)
    exps = np.exp(shift_x)
    return exps / np.sum(exps, axis=1, keepdims=True)

class AdamOptimizer:
    def __init__(self, params, lr=1e-3, beta1=0.9, beta2=0.999, eps=1e-8):
        self.params = params
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.m = {k: np.zeros_like(v) for k, v in params.items()}
        self.v = {k: np.zeros_like(v) for k, v in params.items()}
        self.t = 0
        
    def step(self, grads, clip_norm=None):
        self.t += 1
        
        # 梯度裁剪 (Gradient Clipping) - 防止 Loss 爆炸的核心
        if clip_norm is not None:
            total_norm = 0
            for k in grads:
                total_norm += np.sum(grads[k]**2)
            total_norm = np.sqrt(total_norm)
            
            if total_norm > clip_norm:
                scale = clip_norm / (total_norm + 1e-8)
                for k in grads:
                    grads[k] *= scale
        
        for k in self.params:
            self.m[k] = self.beta1 * self.m[k] + (1 - self.beta1) * grads[k]
            self.v[k] = self.beta2 * self.v[k] + (1 - self.beta2) * (grads[k]**2)
            
            m_hat = self.m[k] / (1 - self.beta1**self.t)
            v_hat = self.v[k] / (1 - self.beta2**self.t)
            
            self.params[k] -= self.lr * m_hat / (np.sqrt(v_hat) + self.eps)

class WideNet:
    """宽双层网络: Input -> 1024 (Leaky ReLU) -> Output"""
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.params = {}
        # Kaiming Initialization (He Init) - 适合 ReLU 类激活函数
        self.params['W1'] = np.random.randn(input_dim, hidden_dim) * np.sqrt(2.0 / input_dim)
        self.params['b1'] = np.zeros(hidden_dim)
        self.params['W2'] = np.random.randn(hidden_dim, output_dim) * np.sqrt(2.0 / hidden_dim)
        self.params['b2'] = np.zeros(output_dim)

    def forward_loss(self, X, y=None, reg=0.0, dropout_p=0.5, training=True):
        W1, b1 = self.params['W1'], self.params['b1']
        W2, b2 = self.params['W2'], self.params['b2']
        N = X.shape[0]

        # --- Layer 1 ---
        z1 = X.dot(W1) + b1
        a1 = leaky_relu(z1) # 使用 Leaky ReLU 防止神经元死亡
        
        # Dropout
        mask = None
        if training and dropout_p > 0:
            mask = (np.random.rand(*a1.shape) < (1 - dropout_p)) / (1 - dropout_p)
            a1 *= mask

        # --- Layer 2 (Output) ---
        scores = a1.dot(W2) + b2
        
        if y is None: return softmax(scores) # 预测模式返回概率

        # --- Loss Calculation ---
        probs = softmax(scores)
        correct_logprobs = -np.log(probs[np.arange(N), y] + 1e-9)
        data_loss = np.sum(correct_logprobs) / N
        reg_loss = 0.5 * reg * (np.sum(W1**2) + np.sum(W2**2))
        loss = data_loss + reg_loss

        # --- Backward Pass ---
        grads = {}
        dscores = probs
        dscores[np.arange(N), y] -= 1
        dscores /= N

        # Backprop W2, b2
        grads['W2'] = a1.T.dot(dscores) + reg * W2
        grads['b2'] = np.sum(dscores, axis=0)

        # Backprop Layer 1
        da1 = dscores.dot(W2.T)
        if training and dropout_p > 0: da1 *= mask
        
        # Leaky ReLU Gradient
        dz1 = np.ones_like(da1)
        dz1[z1 < 0] = 0.01 # Leaky part
        dz1 = da1 * dz1

        grads['W1'] = X.T.dot(dz1) + reg * W1
        grads['b1'] = np.sum(dz1, axis=0)

        return loss, grads

# ================= 4. 主训练流程 =================

print("正在加载数据...")
X_raw, y_raw, img_shape = load_and_process_data(TRAIN_PATH, has_labels=True)
X_test_raw, _, _ = load_and_process_data(TEST_PATH, has_labels=False)

# 严格的标准化 (Standardization)
# 必须使用 (X - mean) / std，并且记录 mean/std 用于测试集
mean_vals = np.mean(X_raw, axis=0)
std_vals = np.std(X_raw, axis=0) + 1e-8

X_train_norm = (X_raw - mean_vals) / std_vals
X_test_norm = (X_test_raw - mean_vals) / std_vals

print(f"数据加载完毕。输入维度: {X_train_norm.shape[1]}")
print(f"特征均值范围: {np.min(mean_vals):.2f} ~ {np.max(mean_vals):.2f} (应在0-1之间)")

models = []
indices = np.arange(len(X_train_norm))
fold_size = len(X_train_norm) // NUM_MODELS

print(f"\n>>> 开始训练 (Target: Validation Acc > 0.5)")
print(f"    Config: Hidden={HIDDEN_DIM}, LR={LEARNING_RATE}, Clip={GRAD_CLIP}")

for i in range(NUM_MODELS):
    # 划分验证集 (Fold)
    val_idx = indices[i*fold_size : (i+1)*fold_size]
    train_idx = np.concatenate([indices[:i*fold_size], indices[(i+1)*fold_size:]])
    
    X_tr, y_tr = X_train_norm[train_idx], y_raw[train_idx]
    X_val, y_val = X_train_norm[val_idx], y_raw[val_idx]
    
    # 初始化
    net = WideNet(X_train_norm.shape[1], HIDDEN_DIM, NUM_CLASSES)
    opt = AdamOptimizer(net.params, lr=LEARNING_RATE)
    
    # 动态调整轮数
    iters_per_epoch = len(X_tr) // BATCH_SIZE
    total_iters = EPOCHS * iters_per_epoch
    
    best_val_acc = 0.0
    
    print(f"\n[Model {i+1}] Training...")
    
    for it in range(total_iters):
        # 采样
        batch_mask = np.random.choice(len(X_tr), BATCH_SIZE)
        X_batch = X_tr[batch_mask]
        y_batch = y_tr[batch_mask]
        
        # 增强
        X_batch_aug = augment_batch(X_batch, img_shape)
        
        # 训练一步
        loss, grads = net.forward_loss(X_batch_aug, y_batch, reg=L2_REG, dropout_p=0.5)
        
        # 关键修复: 梯度裁剪 !!!
        opt.step(grads, clip_norm=GRAD_CLIP)
        
        if it % 100 == 0:
            # 监控验证集
            val_probs = net.forward_loss(X_val, training=False)
            val_acc = np.mean(np.argmax(val_probs, axis=1) == y_val)
            print(f"Iter {it:4d} | Loss: {loss:.4f} | Val Acc: {val_acc:.4f}", end='\r')
            
    # 最终验证
    val_probs = net.forward_loss(X_val, training=False)
    final_acc = np.mean(np.argmax(val_probs, axis=1) == y_val)
    print(f"\nModel {i+1} Finished. Final Val Acc: {final_acc:.4f}")
    
    if final_acc < 0.25:
        print("警告: 该模型似乎未收敛，建议检查数据或重试。")
    
    models.append(net)

# ================= 5. TTA 预测与集成 =================
print("\n>>> 开始 TTA 预测集成...")

final_logits = np.zeros((len(X_test_norm), NUM_CLASSES))

# 准备测试集翻转版
X_test_flip = X_test_norm.reshape(len(X_test_norm), img_shape[0], img_shape[1], img_shape[2])
X_test_flip = X_test_flip[:, :, :, ::-1].reshape(len(X_test_norm), -1)

for idx, net in enumerate(models):
    # 原图预测
    p1 = net.forward_loss(X_test_norm, training=False)
    # 翻转图预测
    p2 = net.forward_loss(X_test_flip, training=False)
    
    # 累加概率
    final_logits += (p1 + p2) / 2.0

final_preds = np.argmax(final_logits, axis=1)

# 生成提交
filename = 'submission_stable_ultimate.csv'
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ID', 'Label'])
    for i, label in enumerate(final_preds):
        writer.writerow([i + 1, label])

print(f"\n生成文件: {filename}")
print("分析: 通过梯度裁剪(Gradient Clipping)和宽网络结构，Loss 应该被控制在 2.0 以内，Validation Acc 有望突破 0.5。")

>>> 环境配置完成。启用 'Stable Ultimate' 模式。
正在加载数据...
数据加载完毕。输入维度: 2352
特征均值范围: 0.00 ~ 0.45 (应在0-1之间)

>>> 开始训练 (Target: Validation Acc > 0.5)
    Config: Hidden=1024, LR=0.0005, Clip=1.0

[Model 1] Training...
Iter 1000 | Loss: 1405.1479 | Val Acc: 0.4163
Model 1 Finished. Final Val Acc: 0.4034

[Model 2] Training...
Iter 1000 | Loss: 1422.8689 | Val Acc: 0.3250
Model 2 Finished. Final Val Acc: 0.1974
警告: 该模型似乎未收敛，建议检查数据或重试。

[Model 3] Training...
Iter 1000 | Loss: 1443.5071 | Val Acc: 0.3093
Model 3 Finished. Final Val Acc: 0.3509

[Model 4] Training...
Iter 1000 | Loss: 1429.6337 | Val Acc: 0.1780
Model 4 Finished. Final Val Acc: 0.1823
警告: 该模型似乎未收敛，建议检查数据或重试。

[Model 5] Training...
Iter 1000 | Loss: 1393.7950 | Val Acc: 0.2797
Model 5 Finished. Final Val Acc: 0.1857
警告: 该模型似乎未收敛，建议检查数据或重试。

>>> 开始 TTA 预测集成...

生成文件: submission_stable_ultimate.csv
分析: 通过梯度裁剪(Gradient Clipping)和宽网络结构，Loss 应该被控制在 2.0 以内，Validation Acc 有望突破 0.5。
